In [1]:
import requests
import pandas as pd
from google.cloud import bigquery
import json

ModuleNotFoundError: No module named 'google.cloud'

# Load API Data into DataFrame

In [15]:
url = "https://zillow56.p.rapidapi.com/search"

querystring = {"location":"30078","output":"json","status":"forSale","sortSelection":"priorityscore","listing_type":"by_agent","doz":"any"}

headers = {
	"x-rapidapi-key": "ee7c8ac8bdmsh0758d3dc8c293efp13865bjsne224cba16dd3",
	"x-rapidapi-host": "zillow56.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
data = response.json()

In [16]:
raw_data = pd.DataFrame(data['results'])

In [17]:
querystring = {"location":"30078",'page':str(2),"output":"json","status":"forSale","sortSelection":"priorityscore","listing_type":"by_agent","doz":"any"}
response = requests.get(url, headers=headers, params=querystring)
data = response.json()
raw_data = pd.concat([raw_data, pd.DataFrame(data['results'])])

In [18]:
querystring = {"location":"30078",'page':str(3),"output":"json","status":"forSale","sortSelection":"priorityscore","listing_type":"by_agent","doz":"any"}
response = requests.get(url, headers=headers, params=querystring)
data = response.json()
raw_data = pd.concat([raw_data, pd.DataFrame(data['results'])])

In [19]:
querystring = {"location":"30078",'page':str(4),"output":"json","status":"forSale","sortSelection":"priorityscore","listing_type":"by_agent","doz":"any"}
response = requests.get(url, headers=headers, params=querystring)
data = response.json()
raw_data = pd.concat([raw_data, pd.DataFrame(data['results'])])

# Load Raw Data into BigQuery Table

In [21]:
client = bigquery.Client('db-management-final-project')
dataset = client.dataset('zillow_dataset')
bq_table = client.get_table('db-management-final-project.zillow_dataset.raw_zip_30078')

In [22]:
schema = [bigquery.SchemaField("city", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("country", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("currency", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("homeStatus", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("homeStatusForHDP", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("homeType", bigquery.enums.SqlTypeNames.STRING), 
          bigquery.SchemaField("imgSrc", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("lotAreaUnit", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("priceReduction", bigquery.enums.SqlTypeNames.STRING), 
          bigquery.SchemaField("state", bigquery.enums.SqlTypeNames.STRING), 
          bigquery.SchemaField("streetAddress", bigquery.enums.SqlTypeNames.STRING), 
          bigquery.SchemaField("zipcode", bigquery.enums.SqlTypeNames.STRING), 
          bigquery.SchemaField("unit", bigquery.enums.SqlTypeNames.STRING)]

job_config = bigquery.LoadJobConfig(schema=schema,
                                    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE)

job = client.load_table_from_dataframe(table, 'db-management-final-project.zillow_dataset.raw_zip_30078', job_config=job_config)
job.result()

LoadJob<project=db-management-final-project, location=US, id=aa8408f6-4c50-4fe3-9980-e4adc331ba3d>

# Data Transformation and Validation

In [25]:
table = pd.concat([raw_data.reset_index(drop = True), pd.json_normalize(raw_data['listing_sub_type'])], axis = 1)
table

,bathrooms,bedrooms,city,country,currency,datePriceChanged,daysOnZillow,homeStatus,homeStatusForHDP,homeType,...,newConstructionType,comingSoonOnMarketDate,providerListingID,videoCount,group_type,priceSuffix,is_FSBA,is_openHouse,is_newHome,is_comingSoon
0,3.0,4.0,Snellville,USA,USD,1.732262e+12,-1,FOR_SALE,FOR_SALE,SINGLE_FAMILY,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
1,2.0,3.0,Snellville,USA,USD,1.730966e+12,-1,FOR_SALE,FOR_SALE,SINGLE_FAMILY,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
2,4.0,5.0,Snellville,USA,USD,1.732694e+12,-1,FOR_SALE,FOR_SALE,SINGLE_FAMILY,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
3,2.0,4.0,Snellville,USA,USD,NaN,-1,FOR_SALE,FOR_SALE,SINGLE_FAMILY,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
4,3.0,4.0,Snellville,USA,USD,1.732694e+12,-1,FOR_SALE,FOR_SALE,SINGLE_FAMILY,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,0.0,0.0,Snellville,USA,USD,NaN,-1,FOR_SALE,FOR_SALE,LOT,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
147,0.0,0.0,Snellville,USA,USD,NaN,-1,FOR_SALE,FOR_SALE,LOT,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
148,NaN,NaN,Snellville,USA,USD,1.692169e+12,-1,FOR_SALE,FOR_SALE,LOT,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN
149,0.0,0.0,Snellville,USA,USD,1.692169e+12,-1,FOR_SALE,FOR_SALE,LOT,...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN


In [26]:
# drop null columns
table = table.drop(['newConstructionType', 'openHouse', 'open_house_info', 'videoCount', 'comingSoonOnMarketDate', 'listing_sub_type', 'providerListingID', 'group_type', 'priceSuffix'], axis = 1)

In [27]:
# impute null values in remaining columns
table['is_FSBA'] = table['is_FSBA'].fillna(False)
table['is_newHome'] = table['is_newHome'].fillna(False)
table['is_openHouse'] = table['is_openHouse'].fillna(False)
table['is_comingSoon'] = table['is_comingSoon'].fillna(False)
table['bathrooms'] = table['bathrooms'].fillna(table['bathrooms'].mean())
table['bedrooms'] = table['bedrooms'].fillna(table['bedrooms'].mean())
table['datePriceChanged'] = table['datePriceChanged'].fillna(0)
table['livingArea'] = table['livingArea'].fillna(table['livingArea'].mean())
table['lotAreaUnit'] = table['lotAreaUnit'].fillna('acres')
table['lotAreaValue'] = table['lotAreaValue'].fillna(table['lotAreaValue'].mean())
table['priceChange'] = table['priceChange'].fillna(0)
table['priceReduction'] = table['priceReduction'].fillna(0)
table['rentZestimate'] = table['rentZestimate'].fillna(table['rentZestimate'].mean())
table['taxAssessedValue'] = table['taxAssessedValue'].fillna(table['taxAssessedValue'].mean())
table['zestimate'] = table['zestimate'].fillna(table['zestimate'].mean())
table['unit'] = table['unit'].fillna('None')

In [28]:
# save data to csv to prevent making numerous API calls
table.to_csv('zillow_dataset.csv')
table = pd.read_csv('zillow_dataset.csv', index_col = 0)

# Load Cleaned Data into BigQuery Table

In [30]:
bq_table = client.get_table('db-management-final-project.zillow_dataset.zip_30078')

schema = [bigquery.SchemaField("city", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("country", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("currency", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("homeStatus", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("homeStatusForHDP", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("homeType", bigquery.enums.SqlTypeNames.STRING), 
          bigquery.SchemaField("imgSrc", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("lotAreaUnit", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("priceReduction", bigquery.enums.SqlTypeNames.STRING), 
          bigquery.SchemaField("state", bigquery.enums.SqlTypeNames.STRING), 
          bigquery.SchemaField("streetAddress", bigquery.enums.SqlTypeNames.STRING), 
          bigquery.SchemaField("zipcode", bigquery.enums.SqlTypeNames.STRING), 
          bigquery.SchemaField("unit", bigquery.enums.SqlTypeNames.STRING)]

job_config = bigquery.LoadJobConfig(schema=schema,
                                    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE)

job = client.load_table_from_dataframe(table, 'db-management-final-project.zillow_dataset.zip_30078', job_config=job_config)
job.result()

LoadJob<project=db-management-final-project, location=US, id=eda71e2a-a0ba-48fc-8269-93d87f7b896c>

# Run Queries on Transformed Data

In [31]:
query = 'select * from db-management-final-project.zillow_dataset.zip_30078'
query_job = client.query(query)  # API request
rows = query_job.result()
query_job.to_dataframe()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,bathrooms,bedrooms,city,country,currency,datePriceChanged,daysOnZillow,homeStatus,homeStatusForHDP,homeType,...,taxAssessedValue,timeOnZillow,zestimate,zipcode,zpid,unit,is_FSBA,is_openHouse,is_newHome,is_comingSoon
0,0.000000,0.000000,Snellville,USA,USD,0.000000e+00,-1,FOR_SALE,FOR_SALE,LOT,...,374666.071429,8557264000,461226.229508,30078,2054087885,# 49,True,False,False,False
1,0.000000,0.000000,Snellville,USA,USD,0.000000e+00,-1,FOR_SALE,FOR_SALE,LOT,...,374666.071429,63248464000,461226.229508,30078,2078873608,Lot 25,True,False,False,False
2,0.000000,0.000000,Snellville,USA,USD,0.000000e+00,-1,FOR_SALE,FOR_SALE,LOT,...,374666.071429,6829264000,461226.229508,30078,440376282,# 9-10,True,False,False,False
3,0.000000,0.000000,Snellville,USA,USD,0.000000e+00,-1,FOR_SALE,FOR_SALE,LOT,...,374666.071429,17024464000,461226.229508,30078,351225726,# 15,True,False,False,False
4,0.000000,0.000000,Snellville,USA,USD,0.000000e+00,-1,FOR_SALE,FOR_SALE,LOT,...,374666.071429,24886864000,461226.229508,30078,340152266,# 3,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,2.838028,3.429577,Snellville,USA,USD,0.000000e+00,-1,FOR_SALE,FOR_SALE,LOT,...,374666.071429,7693264000,461226.229508,30078,440007818,None,True,False,False,False
147,2.838028,3.429577,Snellville,USA,USD,0.000000e+00,-1,FOR_SALE,FOR_SALE,LOT,...,45000.000000,4669264000,461226.229508,30078,440909678,None,True,False,False,False
148,2.838028,3.429577,Snellville,USA,USD,1.681888e+12,-1,FOR_SALE,FOR_SALE,LOT,...,191000.000000,46054864000,461226.229508,30078,14704144,None,True,False,False,False
149,2.838028,3.429577,Snellville,USA,USD,1.692169e+12,-1,FOR_SALE,FOR_SALE,LOT,...,698600.000000,58669264000,461226.229508,30078,172201967,None,True,False,False,False
